## Some simple data explorations - Part 1
### Things to be noted first:
- Binary classification problems
- Imbalanced dataset

### Number of records for each file

In [3]:
import pandas as pd

train_user = pd.read_csv('../../ml_case_data/ml_case_training_data.csv')
train_hist = pd.read_csv('../../ml_case_data/ml_case_training_hist_data.csv')
train_churn = pd.read_csv('../../ml_case_data/ml_case_training_output.csv')

#print (len(train_user), len(train_hist), len(train_churn))
train_user.columns

Index(['id', 'activity_new', 'campaign_disc_ele', 'channel_sales', 'cons_12m',
       'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end',
       'date_first_activ', 'date_modif_prod', 'date_renewal',
       'forecast_base_bill_ele', 'forecast_base_bill_year',
       'forecast_bill_12m', 'forecast_cons', 'forecast_cons_12m',
       'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_p1',
       'forecast_price_energy_p2', 'forecast_price_pow_p1', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max'],
      dtype='object')

### Split the files into class 0 and 1 (No churn and Churn)

In [4]:
churn_id = list(set(list(train_churn[train_churn.churn==1]['id'])))

In [5]:
usr_info_0 = train_user[~train_user.id.isin(churn_id)]
usr_info_1 = train_user[train_user.id.isin(churn_id)]
hist_0 = train_hist[~train_hist.id.isin(churn_id)]
hist_1 = train_hist[train_hist.id.isin(churn_id)]

In [6]:
len(usr_info_1.id.unique())

1595

### Take a look at null values in each column, for each class:
- I intend to filter out some columns with many null values (>50%)
- *activity_new, campaign_disc_ele, date_first_activ, and some forecast_*\* columns will be dropped

In [14]:
import numpy as np

temp0 = []
temp1 = []
for c in list(train_user.columns):
    temp0.append(len(usr_info_0[usr_info_0[c].notnull()]))
    temp1.append(len(usr_info_1[usr_info_1[c].notnull()]))

print ('Columns, Not_churn_client, Churn_client')
for i in range(len(temp0)):
    print (list(train_user.columns)[i], ': ', str(temp0[i]), '/14501, ', str(temp1[i]), '/1595.')

Columns, Not_churn_client, Churn_client
id :  14501 /14501,  1595 /1595.
activity_new :  5912 /14501,  639 /1595.
campaign_disc_ele :  0 /14501,  0 /1595.
channel_sales :  10608 /14501,  1270 /1595.
cons_12m :  14501 /14501,  1595 /1595.
cons_gas_12m :  14501 /14501,  1595 /1595.
cons_last_month :  14501 /14501,  1595 /1595.
date_activ :  14501 /14501,  1595 /1595.
date_end :  14499 /14501,  1595 /1595.
date_first_activ :  2977 /14501,  531 /1595.
date_modif_prod :  14369 /14501,  1570 /1595.
date_renewal :  14461 /14501,  1595 /1595.
forecast_base_bill_ele :  2977 /14501,  531 /1595.
forecast_base_bill_year :  2977 /14501,  531 /1595.
forecast_bill_12m :  2977 /14501,  531 /1595.
forecast_cons :  2977 /14501,  531 /1595.
forecast_cons_12m :  14501 /14501,  1595 /1595.
forecast_cons_year :  14501 /14501,  1595 /1595.
forecast_discount_energy :  14421 /14501,  1549 /1595.
forecast_meter_rent_12m :  14501 /14501,  1595 /1595.
forecast_price_energy_p1 :  14421 /14501,  1549 /1595.
forecas

### Next, check statistical info about the two classes:

In [15]:
usr_info_0.describe()

,campaign_disc_ele,cons_12m,cons_gas_12m,cons_last_month,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,...,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max
count,0.0,1.450100e+04,1.450100e+04,1.450100e+04,2977.000000,2977.000000,2977.000000,2977.000000,14501.000000,14501.000000,...,14421.000000,14421.000000,14421.000000,14501.000000,14488.000000,14488.000000,14501.000000,14486.000000,14501.000000,14498.000000
mean,NaN,2.064686e+05,3.431931e+04,2.074335e+04,335.725139,335.725139,3821.612518,207.854716,2360.659598,1902.542032,...,0.135929,0.052543,43.526605,195.563286,21.831935,20.875035,1.358734,214.420518,5.070409,20.535843
std,NaN,7.084568e+05,1.860753e+05,8.614061e+04,660.726144,660.726144,5342.475631,462.549998,3985.065460,5001.109911,...,0.026317,0.048548,5.325953,475.032779,22.580035,26.928157,1.525549,307.624318,1.679062,21.948308
min,NaN,-1.169570e+05,-3.037000e+03,-9.138600e+04,-364.940000,-364.940000,-2503.480000,0.000000,-16689.260000,-85627.000000,...,0.000000,0.000000,-0.122184,-9038.210000,-525.540000,-615.660000,1.000000,-4148.990000,1.000000,1.000000
25%,NaN,5.918000e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,1137.580000,0.000000,514.790000,0.000000,...,0.115237,0.000000,40.606701,0.000000,11.820000,11.590000,1.000000,51.912500,4.000000,12.500000
50%,NaN,1.536600e+04,0.000000e+00,9.250000e+02,155.950000,155.950000,2165.530000,40.510000,1178.960000,383.000000,...,0.142881,0.085803,44.311378,44.750000,20.520000,20.430000,1.000000,118.730000,5.000000,13.856000
75%,NaN,5.066200e+04,0.000000e+00,4.209000e+03,395.250000,395.250000,4245.810000,227.510000,2687.160000,2004.000000,...,0.146348,0.098837,44.311378,219.100000,28.600000,28.560000,1.000000,273.365000,6.000000,19.800000
max,NaN,1.609711e+07,4.188440e+06,4.538720e+06,12566.080000,12566.080000,61993.560000,9682.890000,103801.930000,175375.000000,...,0.273963,0.195975,59.444710,15042.790000,374.640000,374.640000,32.000000,10203.500000,16.000000,500.000000


In [16]:
usr_info_1.describe()

,campaign_disc_ele,cons_12m,cons_gas_12m,cons_last_month,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,...,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max
count,0.0,1.595000e+03,1595.000000,1595.000000,531.000000,531.000000,531.000000,531.000000,1595.000000,1595.000000,...,1549.000000,1549.000000,1549.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000
mean,NaN,8.875863e+04,10022.208150,7807.845768,336.509435,336.509435,3926.187571,201.185217,2460.528978,1951.033856,...,0.135633,0.056748,43.597652,201.216169,28.187912,26.776665,1.248276,250.378539,4.668966,21.224839
std,NaN,2.900448e+05,54312.082719,28919.448358,582.448732,582.448732,5875.351406,415.091819,4464.574325,7181.095468,...,0.025646,0.049115,4.002373,644.218217,31.577247,35.235577,0.579709,704.058290,1.604810,20.099010
min,NaN,-1.252760e+05,-617.000000,-11825.000000,0.000000,0.000000,414.540000,0.000000,-10037.370000,-19057.000000,...,0.092453,0.000000,32.583108,-1824.080000,-341.220000,-341.220000,1.000000,-3711.400000,2.000000,3.300000
25%,NaN,5.836000e+03,0.000000,0.000000,0.000000,0.000000,1222.320000,0.000000,500.740000,0.000000,...,0.115237,0.000000,40.606701,0.000000,15.500000,15.445000,1.000000,52.605000,4.000000,13.150000
50%,NaN,1.513400e+04,0.000000,747.000000,189.280000,189.280000,2430.180000,52.600000,1188.800000,347.000000,...,0.141666,0.088444,44.311378,42.400000,25.840000,25.800000,1.000000,129.320000,4.000000,14.490000
75%,NaN,4.620250e+04,0.000000,3471.000000,396.205000,396.205000,4294.075000,230.375000,2760.105000,1912.500000,...,0.145711,0.099419,44.311378,211.355000,34.500000,34.440000,1.000000,297.545000,6.000000,20.700000
max,NaN,4.012564e+06,828344.000000,469898.000000,5432.440000,5432.440000,81122.630000,4205.640000,66521.260000,155432.000000,...,0.273963,0.168092,59.266378,13001.820000,299.640000,299.640000,5.000000,24570.650000,13.000000,320.000000


So for the two tables above, I did not see great differences between the two classes except for net margin and gross margin

In [17]:
hist_0.describe()

,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
count,172712.000000,172712.000000,172712.000000,172712.000000,172712.000000,172712.000000
mean,0.140960,0.054017,0.030249,43.310810,10.552225,6.353251
std,0.025191,0.049964,0.036234,5.561288,12.843232,7.758092
min,0.000000,0.000000,0.000000,-0.177779,-0.097752,-0.065172
25%,0.125976,0.000000,0.000000,40.728885,0.000000,0.000000
50%,0.146517,0.085058,0.000000,44.266930,0.000000,0.000000
75%,0.151635,0.101673,0.072394,44.444710,24.339581,16.226389
max,0.280700,0.229788,0.114102,59.444710,36.490692,17.458221


In [18]:
hist_1.describe()

,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
count,18931.000000,18931.000000,18931.000000,18931.000000,18931.000000,18931.000000
mean,0.141279,0.058010,0.034937,43.459985,12.029976,7.387700
std,0.024439,0.050517,0.036983,4.144348,12.896629,7.939266
min,0.000000,0.000000,0.000000,-0.162916,-0.097752,-0.065172
25%,0.125165,0.000000,0.000000,40.728885,0.000000,0.000000
50%,0.144673,0.087970,0.000000,44.266930,0.000000,0.000000
75%,0.151367,0.103059,0.073433,44.444710,24.437330,16.226389
max,0.280700,0.169597,0.101037,59.444710,36.490689,16.958222


### Drop columns with null values over 50% and save to 'test.csv'

In [4]:
filtered = train_user[train_user.date_end.notnull() & \
                      train_user.date_modif_prod.notnull() & \
                      train_user.date_renewal.notnull() & \
                      train_user.forecast_discount_energy.notnull() & \
                      train_user.forecast_price_energy_p1.notnull() & \
                      train_user.forecast_price_energy_p2.notnull() & \
                      train_user.forecast_price_pow_p1.notnull() & \
                      train_user.margin_gross_pow_ele.notnull() & \
                      train_user.margin_net_pow_ele.notnull() & \
                      train_user.net_margin.notnull() & \
                      train_user.origin_up.notnull()]

In [5]:
fil_cols = ['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'cons_last_month', \
            'date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m', \
            'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', \
            'forecast_price_energy_p1', 'forecast_price_energy_p2', 'forecast_price_pow_p1', \
            'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', \
            'net_margin', 'num_years_antig', 'origin_up', 'pow_max']
fil_df = filtered[fil_cols].fillna('nodata')

In [7]:
fil_df.to_csv('./test.csv', index=False)